In [ ]:
# Datahandling
import os
import re
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.sparse
import matplotlib.pyplot as plt

# Maskinlæring
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report)
from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold, mutual_info_classif
import xgboost as xgb

# Andre biblioteker
import joblib


## SetFit

In [ ]:
!pip install setfit datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install transformers==4.45.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [ ]:
# Dropbox download
url = "https://www.dropbox.com/scl/fi/sdsekerjmi86a42yj7eb0/TwitterData_Joined.csv?rlkey=cvtelhf5yrv9uy4f7fma8du7a&st=o8c18q94&dl=1"

response = requests.get(url)
with open("TwitterData_Joined.csv", "wb") as f:
    f.write(response.content)

In [ ]:
# DATA IMPORT & SAMPLING
data = pd.read_csv('TwitterData_Joined.csv')
data = data.sample(n=2000, random_state=19)

## 500 forsøg

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset

# DATA IMPORT & SAMPLING
data = pd.read_csv('TwitterData_Joined.csv')
data = data.sample(n=500, random_state=19)

# Konverter pandas DataFrame til Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Split datasættet i træning og test
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_ds = dataset["train"]
test_ds = dataset["test"]

# Indlæs en prætrænet SetFit-model
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    batch_size=8,  # Reducer fra 32 til 8
    num_iterations=10,
    num_epochs=1,
    column_mapping={"Tweet_text": "text", "Label": "label"}
)

# Træn modellen
trainer.train()

# Evaluer modellen
metrics = trainer.evaluate()
print(metrics)  # Udskriv præcision, recall og F1-score


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
<ipython-input-5-053dc6f1127d>:16: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 8000
  Batch size = 8
  Num epochs = 1
/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: adamsen (adamsen-aalborg-universitet). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,0.442000
50,0.290300
100,0.246200
150,0.250700
200,0.249300
250,0.230300
300,0.183100
350,0.102400
400,0.054300
450,0.030100


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.81}


### Forsøg på mindre model

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset

# DATA IMPORT & SAMPLING
data = pd.read_csv('TwitterData_Joined.csv')
data = data.sample(n=500, random_state=19)


# Konverter pandas DataFrame til Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Split datasættet i træning og test
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_ds = dataset["train"]
test_ds = dataset["test"]

# Indlæs en prætrænet SetFit-model
model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    batch_size=8,  # Reducer fra 32 til 8
    num_iterations=10,
    num_epochs=1,
    column_mapping={"Tweet_text": "text", "Label": "label"}
)

# Træn modellen
trainer.train()

# Evaluer modellen
metrics = trainer.evaluate()
print(metrics)  # Udskriv præcision, recall og F1-score


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,0.614800
50,0.366900
100,0.260300
150,0.247700
200,0.247400
250,0.246900
300,0.230200
350,0.213000
400,0.178400
450,0.144300


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.78}


## 16 batch

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset

# DATA IMPORT & SAMPLING
data = pd.read_csv('TwitterData_Joined.csv')
data = data.sample(n=500, random_state=19)


# Konverter pandas DataFrame til Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Split datasættet i træning og test
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_ds = dataset["train"]
test_ds = dataset["test"]

# Indlæs en prætrænet SetFit-model
model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    batch_size=16,
    num_iterations=10,
    num_epochs=1,
    column_mapping={"Tweet_text": "text", "Label": "label"}
)

# Træn modellen
trainer.train()

# Evaluer modellen
metrics = trainer.evaluate()
print(metrics)  # Udskriv præcision, recall og F1-score


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
<ipython-input-12-67b4d0ec9605>:21: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 8000
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.526000
50,0.325400
100,0.248300
150,0.246000
200,0.234400
250,0.201300
300,0.170500
350,0.135400
400,0.111200
450,0.086500


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.77}


## 2000 forsøg:

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset

data = pd.read_csv('TwitterData_Joined.csv')
data = data.sample(n=2000, random_state=19)

# Konverter pandas DataFrame til Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Split datasættet i træning og test
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_ds = dataset["train"]
test_ds = dataset["test"]

# Indlæs en prætrænet SetFit-model
model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    batch_size=8,  # Reducer fra 32 til 8
    num_iterations=10,
    num_epochs=1,
    column_mapping={"Tweet_text": "text", "Label": "label"}
)

# Træn modellen
trainer.train()

# Evaluer modellen
metrics = trainer.evaluate()
print(metrics)  # Udskriv præcision, recall og F1-score


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
<ipython-input-11-1d0f484ac205>:16: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 32000
  Batch size = 8
  Num epochs = 1


Step,Training Loss
1,0.528300
50,0.361000
100,0.332400
150,0.272300
200,0.263400
250,0.253500
300,0.251300
350,0.246100
400,0.251700
450,0.236900


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.8475}


## 5000 forsøg:

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset

data = pd.read_csv('TwitterData_Joined.csv')
data = data.sample(n=5000, random_state=19)

# Konverter pandas DataFrame til Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Split datasættet i træning og test
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_ds = dataset["train"]
test_ds = dataset["test"]

# Indlæs en prætrænet SetFit-model
model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    batch_size=8,  # Reducer fra 32 til 8
    num_iterations=10,
    num_epochs=1,
    column_mapping={"Tweet_text": "text", "Label": "label"}
)

# Træn modellen
trainer.train()

# Evaluer modellen
metrics = trainer.evaluate()
print(metrics)  # Udskriv præcision, recall og F1-score


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
<ipython-input-13-69c9ece4c81e>:19: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 80000
  Batch size = 8
  Num epochs = 1


Step,Training Loss
1,0.540000
50,0.383900
100,0.392000
150,0.338800
200,0.280500
250,0.264400
300,0.258200
350,0.260200
400,0.253700
450,0.249700


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.864}


## 10000 forsøg

In [5]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset

data = pd.read_csv('TwitterData_Joined.csv')
data = data.sample(n=10000, random_state=19)

# Konverter pandas DataFrame til Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Split datasættet i træning og test
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_ds = dataset["train"]
test_ds = dataset["test"]

# Indlæs en prætrænet SetFit-model
model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    batch_size=8,  # Reducer fra 32 til 8
    num_iterations=10,
    num_epochs=1,
    column_mapping={"Tweet_text": "text", "Label": "label"}
)

# Træn modellen
trainer.train()

# Evaluer modellen
metrics = trainer.evaluate()
print(metrics)  # Udskriv præcision, recall og F1-score

model.save_pretrained("10000_setfit_model")
print("Model saved!")


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,0.496100
50,0.372600
100,0.393300
150,0.336200
200,0.324400
250,0.298200
300,0.272800
350,0.254500
400,0.251700
450,0.261200


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.906}
Model saved!


In [6]:
!zip -r /content/10000_setfit_model.zip /content/10000_setfit_model

  adding: content/10000_setfit_model/ (stored 0%)
  adding: content/10000_setfit_model/tokenizer_config.json (deflated 74%)
  adding: content/10000_setfit_model/config_sentence_transformers.json (deflated 35%)
  adding: content/10000_setfit_model/special_tokens_map.json (deflated 80%)
  adding: content/10000_setfit_model/modules.json (deflated 62%)
  adding: content/10000_setfit_model/tokenizer.json (deflated 71%)
  adding: content/10000_setfit_model/2_Normalize/ (stored 0%)
  adding: content/10000_setfit_model/1_Pooling/ (stored 0%)
  adding: content/10000_setfit_model/1_Pooling/config.json (deflated 57%)
  adding: content/10000_setfit_model/README.md (deflated 79%)
  adding: content/10000_setfit_model/model.safetensors (deflated 8%)
  adding: content/10000_setfit_model/config.json (deflated 47%)
  adding: content/10000_setfit_model/model_head.pkl (deflated 8%)
  adding: content/10000_setfit_model/vocab.txt (deflated 53%)
  adding: content/10000_setfit_model/sentence_bert_config.json 

### 15000 SETFIT TEST (FAILS) - MAX ER 10.000

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset

data = pd.read_csv('TwitterData_Joined.csv')
data = data.sample(n=12000, random_state=19)

# Konverter pandas DataFrame til Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Split datasættet i træning og test
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_ds = dataset["train"]
test_ds = dataset["test"]

# Indlæs en prætrænet SetFit-model
model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    batch_size=8,  # Reducer fra 32 til 8
    num_iterations=10,
    num_epochs=1,
    column_mapping={"Tweet_text": "text", "Label": "label"}
)

# Træn modellen
trainer.train()

# Evaluer modellen
metrics = trainer.evaluate()
print(metrics)  # Udskriv præcision, recall og F1-score

model.save_pretrained("15000_setfit_model")
print("Model saved!")


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
<ipython-input-8-377557892c1c>:19: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

KeyboardInterrupt: 

### K-FOLD SETFIT - 10000

In [ ]:
import os
import pandas as pd
from datasets import Dataset
from setfit import SetFitModel, SetFitTrainer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold
import numpy as np

# Indlæs data
data = pd.read_csv('TwitterData_Joined.csv')
data = data.sample(n=10000, random_state=19)

# Definer K-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
results = []

# Opret mappe til gemte modeller
model_dir = "setfit_models"
os.makedirs(model_dir, exist_ok=True)

# Loop over K-Folds
for fold, (train_idx, test_idx) in enumerate(kfold.split(data)):
    print(f"\nFold {fold + 1}/{kfold.get_n_splits()}")

    # Split data
    train_split = Dataset.from_pandas(data.iloc[train_idx])
    test_split = Dataset.from_pandas(data.iloc[test_idx])

    # Indlæs prætrænet SetFit-model
    model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

    # Opsæt træner
    trainer = SetFitTrainer(
        model=model,
        train_dataset=train_split,
        eval_dataset=test_split,
        batch_size=8,
        num_iterations=10,
        num_epochs=1,
        column_mapping={"Tweet_text": "text", "Label": "label"}
    )

    # Træn modellen
    trainer.train()

    # Gem modellen for denne fold
    fold_model_path = os.path.join(model_dir, f"model_fold_{fold + 1}")
    model.save_pretrained(fold_model_path)
    print(f"Model for Fold {fold + 1} gemt i: {fold_model_path}")

    # Evaluer modellen
    y_true = test_split["Label"]  # Match kolonnenavne her
    y_pred = trainer.model.predict(test_split["Tweet_text"])  # Match kolonnenavne her

    # Beregn metrics
    accuracy = accuracy_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=["Not Bot", "Bot"], output_dict=True)

    # Gem resultater for fold
    results.append({
        "accuracy": accuracy,
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1": report["weighted avg"]["f1-score"]
    })

    # Print metrics for denne fold
    print(f"Accuracy: {accuracy}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=["Not Bot", "Bot"]))

# Beregn gennemsnit for alle folds
avg_accuracy = np.mean([r["accuracy"] for r in results])
avg_precision = np.mean([r["precision"] for r in results])
avg_recall = np.mean([r["recall"] for r in results])
avg_f1 = np.mean([r["f1"] for r in results])

# Udskriv gennemsnit
print("\n--- Gennemsnit for alle folds ---")
print(f"Average Accuracy: {avg_accuracy}")
print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1-Score: {avg_f1}")



Fold 1/5


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
<ipython-input-5-de56f309431e>:33: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 160000
  Batch size = 8
  Num epochs = 1


Step,Training Loss
1,0.498400
50,0.361900
100,0.389600
150,0.333100
200,0.324700
250,0.294400
300,0.287300
350,0.261300
400,0.275100
450,0.248600


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [ ]:
    # Evaluer modellen
    y_true = test_split["Label"]  # Match kolonnenavne her
    y_pred = trainer.model.predict(test_split["Tweet_text"])  # Match kolonnenavne her

    # Beregn metrics
    accuracy = accuracy_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=["Not Bot", "Bot"], output_dict=True)

    # Gem resultater for fold
    results.append({
        "accuracy": accuracy,
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1": report["weighted avg"]["f1-score"]
    })

    # Print metrics for denne fold
    print(f"Accuracy: {accuracy}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=["Not Bot", "Bot"]))

# Beregn gennemsnit for alle folds
avg_accuracy = np.mean([r["accuracy"] for r in results])
avg_precision = np.mean([r["precision"] for r in results])
avg_recall = np.mean([r["recall"] for r in results])
avg_f1 = np.mean([r["f1"] for r in results])

# Udskriv gennemsnit
print("\n--- Gennemsnit for alle folds ---")
print(f"Average Accuracy: {avg_accuracy}")
print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1-Score: {avg_f1}")

Accuracy: 0.9015
Confusion Matrix:
[[842  88]
 [109 961]]
Classification Report:
              precision    recall  f1-score   support

     Not Bot       0.89      0.91      0.90       930
         Bot       0.92      0.90      0.91      1070

    accuracy                           0.90      2000
   macro avg       0.90      0.90      0.90      2000
weighted avg       0.90      0.90      0.90      2000


--- Gennemsnit for alle folds ---
Average Accuracy: 0.9015
Average Precision: 0.9018226311373607
Average Recall: 0.9015
Average F1-Score: 0.9015617565084791


## RAM RESET:

In [ ]:
import gc

# Metode 1: Brug garbage collector
gc.collect()

# Metode 2: Slet variabler manuelt
# Hvis du har en stor variabel kaldet 'large_variable':
del large_variable
gc.collect()

# Metode 3: Brug 'del' med lister
# Hvis du har en stor liste kaldet 'my_list':
my_list = [] # Tøm listen ved at tildele en tom liste
# eller
del my_list[:] # Slet alle elementer i listen
gc.collect()

NameError: name 'large_variable' is not defined

# GPU< test

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
from setfit import SetFitModel, SetFitTrainer
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


class SetFitONNXPipeline:
    def __init__(self,
                 model_name="sentence-transformers/paraphrase-mpnet-base-v2",
                 batch_size=16,
                 num_iterations=20,
                 num_epochs=1):

        # Parametre
        self.model_name = model_name
        self.batch_size = batch_size
        self.num_iterations = num_iterations
        self.num_epochs = num_epochs

        # Initiér modellen
        self.setfit_model = SetFitModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Sikre GPU-tilgængelighed
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Kører på: {self.device}")


    def prepare_dataset(self, data):
        # Konverter pandas DataFrame til Hugging Face Dataset
        dataset = Dataset.from_pandas(data)
        dataset = dataset.train_test_split(test_size=0.2, seed=42)

        return dataset['train'], dataset['test']


    def convert_to_onnx(self):
        # Opret midlertidig mappe til model
        os.makedirs("temp_model", exist_ok=True)

        # Gem den underliggende Hugging Face-model
        self.setfit_model.model.save_pretrained("temp_model")

        # Konverter til ONNX
        onnx_model = ORTModelForSequenceClassification.from_pretrained(
            "temp_model",
            export=True
        )

        # Gem ONNX-model
        os.makedirs("onnx_model", exist_ok=True)
        onnx_model.save_pretrained("onnx_model")

        return onnx_model


    def train_and_evaluate(self, data):
        # Forbered datasæt
        train_ds, test_ds = self.prepare_dataset(data)

        # Opret SetFit Trainer
        trainer = SetFitTrainer(
            model=self.setfit_model,
            train_dataset=train_ds,
            eval_dataset=test_ds,
            batch_size=self.batch_size,
            num_iterations=self.num_iterations,
            num_epochs=self.num_epochs,
            column_mapping={"Tweet_text": "text", "Label": "label"}
        )

        # Træn modellen
        trainer.train()

        # Evaluer modellen
        metrics = trainer.evaluate()
        print("Træningsmetrics:", metrics)

        # Detaljeret klassifikationsrapport
        y_true = [item['label'] for item in test_ds]
        y_pred = trainer.model.predict([item['text'] for item in test_ds])

        print("\nDetaljeret klassifikationsrapport:")
        print(classification_report(y_true, y_pred))

        return trainer


    def onnx_inference(self, new_tweets):
        # Indlæs ONNX-model
        onnx_model = ORTModelForSequenceClassification.from_pretrained("onnx_model")

        # Tokenisér input
        inputs = self.tokenizer(
            new_tweets,
            return_tensors="pt",
            padding=True,
            truncation=True
        )

        # Forudsigelse
        with torch.no_grad():
            outputs = onnx_model(**inputs)
            predictions = outputs.logits.argmax(dim=-1).numpy()

        # Resultat
        for tweet, label in zip(new_tweets, predictions):
            print(f"Tweet: '{tweet}' => Bot-genereret: {label}")

        return predictions


    def full_pipeline(self, data, new_tweets=None):
        # Træn og evaluer
        trainer = self.train_and_evaluate(data)

        # Konverter til ONNX
        onnx_model = self.convert_to_onnx()

        # Inferens på nye tweets (hvis angivet)
        if new_tweets:
            return self.onnx_inference(new_tweets)


# Eksempel på brug
if __name__ == "__main__":
    # Antag at 'data' er dit pandas DataFrame
    pipeline = SetFitONNXPipeline()

    # Kør fuld pipeline
    pipeline.full_pipeline(
        data,
        new_tweets=["Dette er et eksempel tweet", "Endnu et tweet"]
    )

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
<ipython-input-8-8823c777e8ba>:68: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Kører på: cuda


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 3200
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.347400
50,0.244400
100,0.083000
150,0.002000
200,0.000800


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


Træningsmetrics: {'accuracy': 0.55}


KeyError: 'label'

### OLD CODE

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd

# Antager at 'data' allerede er indlæst
# Kolonner, der bruges
columns_to_keep = ['Label', 'Tweet_text']
data = data[columns_to_keep]

# Splitting features og target
X = data['Tweet_text'].fillna("")  # Kun Tweet_text som feature
y = data['Label']  # Target

# Split datasæt
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Konverter trænings- og evalueringsdata til Hugging Face's Dataset-objekt
train_dataset = Dataset.from_dict({"text": X_train.tolist(), "label": y_train.tolist()})
test_dataset = Dataset.from_dict({"text": X_test.tolist(), "label": y_test.tolist()})

# Initialiser en SetFit-model med en pre-trænet Sentence Transformer som base
model_name = "sentence-transformers/paraphrase-mpnet-base-v2"  # Du kan vælge en anden model, hvis ønsket
model = SetFitModel.from_pretrained(model_name)

# Træner SetFit-modellen
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,  # Hugging Face Dataset
    eval_dataset=test_dataset,    # Hugging Face Dataset
    metric="accuracy",  # Standard evalueringsmetrik
    batch_size=16,  # Batch størrelse
    num_iterations=20,  # Antal iterationer i få-skudstræning
)

# Træn modellen
trainer.train()

# Evaluer modellen
metrics = trainer.evaluate()
print("Evalueringsresultater:")
print(metrics)

# Forudsige på testdata
y_test_pred = trainer.model.predict(X_test.tolist())  # Forudsigelser kræver listeformat

# ROC-AUC kræver sandsynligheder, men SetFit returnerer labels direkte
# Derfor bruger vi softmax-aktiveringer for at få sandsynligheder
from scipy.special import softmax
y_test_proba = softmax(trainer.model.model.predict_proba(X_test.tolist()), axis=1)[:, 1]

# Klassifikationsrapport
print("Classification Report:")
print(classification_report(y_test, y_test_pred))
print(f"Test ROC AUC: {roc_auc_score(y_test, y_test_proba):.4f}")

# Gem modellen
model.save_pretrained("setfit_model")
print("SetFit-model gemt!")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
<ipython-input-13-46fd415f7dec>:28: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


AttributeError: 'CallbackHandler' object has no attribute 'tokenizer'

## KeyBERT

In [ ]:
!pip install keybert

In [ ]:
# DATA IMPORT & SAMPLING
data = pd.read_csv('TwitterData_Joined.csv')
data = data.sample(n=10000, random_state=19)

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score
from keybert import KeyBERT
import pandas as pd
import joblib

# Antager at 'data' allerede er indlæst
# Kolonner, der bruges
columns_to_keep = ['Label', 'Tweet_text']
data = data[columns_to_keep]

# Splitting features og target
X = data['Tweet_text'].fillna("")  # Kun Tweet_text som feature
y = data['Label']  # Target

# Brug KeyBERT til nøgleordsudtrækning
keybert_model = KeyBERT()

def extract_keywords(texts, model, top_n=3):
    """Udtrækker nøgleord fra en liste af tekster."""
    keywords_list = []
    for text in texts:
        if not text.strip():  # Håndter tomme tekster
            keywords_list.append("")
        else:
            keywords = model.extract_keywords(text, top_n=top_n, stop_words='english')
            keywords_list.append(" ".join([kw[0] for kw in keywords]))
    return keywords_list

# Generér nøgleord fra Tweet_text
X_keywords = extract_keywords(X.tolist(), keybert_model)

# Vectoriser nøgleord med TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)  # Begræns antallet af features
X_tfidf = vectorizer.fit_transform(X_keywords)

# Split datasæt
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

# Random Forest-model med hyperparameter-tuning
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy')

# Træn modellen
grid_search.fit(X_train, y_train)

# Evaluér på test-sæt
y_test_pred = grid_search.best_estimator_.predict(X_test)
y_test_proba = grid_search.best_estimator_.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_test_pred))
print(f"Test ROC AUC: {roc_auc_score(y_test, y_test_proba):.4f}")

# Gem model og vectorizer
joblib.dump(vectorizer, "vectorizer.pkl")
joblib.dump(grid_search.best_estimator_, "best_model.pkl")
print("Model og vectorizer gemt!")


KeyboardInterrupt: 